In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
import time
from datetime import datetime

/Users/leilayfarsani/Desktop/iati-datastore-/iati_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
api_key = os.getenv('IATI_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please make sure it is set in the .env file or update it if necessary.")

Transactions (Apache-Solr (select))

In [ ]:
import concurrent.futures
import requests
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def fetch_page(start):
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'fl': '*',
        'rows': 1000,
        'start': start
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    
    for attempt in range(5):  # Try up to 5 times
        try:
            response = requests_retry_session().get(base_url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()['response']['docs']
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                wait = 2 ** attempt  # exponential backoff
                print(f"Rate limit hit. Waiting for {wait} seconds.")
                time.sleep(wait)
            else:
                print(f"HTTP error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        
    print(f"Failed to fetch data for start={start} after 5 attempts")
    return []

def get_total_results():
    params = {
        'q': '(sector_code:(11250 OR 12240 OR 31110 OR 31120 OR 31130 OR 31140 OR 31150 OR 31161 OR 31162 OR 31163 OR 31164 OR 31165 OR 31166 OR 31181 OR 31182 OR 31191 OR 31192 OR 31193 OR 31194 OR 31195 OR 31210 OR 31220 OR 31261 OR 31281 OR 31282 OR 31291 OR 31310 OR 31320 OR 31381 OR 31382 OR 31391 OR 32161 OR 32162 OR 43040 OR 43071 OR 43072 OR 43073 OR 52010) OR sector_vocabulary:2 AND sector_code:(311 OR 312 OR 313)) OR (title_narrative:("food security" OR "food insecurity") OR description_narrative:("food security" OR "food insecurity"))',
        'rows': 0
    }
    headers = {'Ocp-Apim-Subscription-Key': api_key}
    response = requests_retry_session().get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()['response']['numFound']
    else:
        print(f"Error: {response.status_code}")
        return 0

base_url = "https://api.iatistandard.org/datastore/transaction/select"
total_results = get_total_results()
all_activities = []

print(f"Total results to fetch: {total_results}")

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:  # Reduced number of workers
    futures = [executor.submit(fetch_page, i) for i in range(0, total_results, 1000)]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        all_activities.extend(future.result())
        print(f"Fetched page {i+1}/{len(futures)}")
        time.sleep(1)  # Add a delay between requests

df_transactions= pd.DataFrame(all_activities)
print(f"Total activities fetched: {len(df_transactions)}")